“Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.”

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
docs=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
documents=text_splitter.split_documents(docs)

documents


In [ ]:
embeddings = OllamaEmbeddings(
    model="embeddinggemma:latest"
)

db = FAISS.from_documents(docs, embeddings)
db


In [ ]:
##querying

query="What is one small, focused action you can take today that will make you slightly better than you were yesterday and quietly shape your future?"
documents=db.similarity_search(query)
documents[0].page_content

##As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [ ]:
retriver=db.as_retriever()
docs=retriver.invoke(query)
docs[0].page_content

Similarity Search with score

There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [ ]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

In [ ]:
embeddings_vector=embeddings.embed_query(query)
embeddings_vector

In [ ]:
docs_score=db.similarity_search_by_vector(embeddings_vector)
docs_score

In [ ]:
#### saving and loading

In [ ]:
db.save_local("faiss_index")

In [ ]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [ ]:
docs